## Sztuczne sieci neuronowe - laboratorium 10

In [1]:
import torch
from torchvision import models

In [2]:
# sprawdzenie, czy GPU jest widoczne
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


## Transfer learning

Dzisiejsze zajęcia będą dotyczyły zagadnienia **transfer learning** - trenowania modeli polegającego na wykorzystaniu architektury i zestawu wag wytrenowanych wcześniej (np. przez kogoś innego - najczęściej badaczy z największych firm, na dużym zbiorze danych), celem wykorzystania "wiedzy" zgromadzonej w już wytrenowanym modelu i przeniesienia jej (stąd "transfer") do innego, zwykle węższego problemu (np. poprzez dotrenowanie na znacznie mniejszym zbiorze danych).

Fazy te nazywają się odpowiednio **pre-training** (tzw. modele pretrenowane, *pretrained models*) i **fine-tuning**.

Wiele z takich gotowych (pretrenowanych) modeli dostępnych jest w pakiecie `torchvision` - części PyTorcha związanej z przetwarzaniem obrazów.

#### Ćwiczenie
Uruchom poniższą komórkę, aby wypisać dostępne w `torchvision` modele. Nazwy modeli zaczynające się dużą literą oznaczają klasy implementujące poszczególne architektury sieci. Ich odpowiedniki pisane małymi literami to funkcje pozwalające zainicjalizować model (https://pytorch.org/vision/stable/models.html).

Funkcje te mają argument `pretrained` - gdy podamy wartość `True`, inicjalizujemy model pretrenowanymi wagami (dla `False` - losowymi).

Wczytaj po kolei wybrane modele (np. `resnet18`) do zmiennej. Sprawdź jej zawartość.

In [3]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [4]:
resnet18_model = models.resnet18(pretrained=True)
print(resnet18_model)

C:\Users\cezar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\cezar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\cezar/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<0

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Klasyfikacja binarna - przygotowanie danych

Będziemy trenować model do klasyfikacji binarnej zdjęć pszczół i mrówek z wykorzystaniem transfer learningu.

Zbiór ten jest dostępny do pobrania tutaj: https://download.pytorch.org/tutorial/hymenoptera_data.zip  
(*hymenoptera* - *błonoskrzydłe* https://pl.wikipedia.org/wiki/B%C5%82onkoskrzyd%C5%82e)

Zbiór ten należy rozpakować do katalogu `common/data` (w razie gdyby go jeszcze tam nie było).

In [6]:
import pathlib

DATA_PATH = pathlib.Path("common/data/hymenoptera_data")

#### Ćwiczenie

Wczytaj zbiór zdjęć (dwa podbiory - train i val) wykorzystując klasę **ImageFolder** dataset dostępną w PyTorch (https://pytorch.org/vision/stable/datasets.html)

Sprawdź rozmiar obu podzbiorów.  
Sprawdź wymiary kilku wybranych zdjęć w zbiorze (używając możliwości klasy `Dataset`, nie przeglądając obrazki w katalogu).

Sprawdź zawartość atrybutów klasy `ImageFolder` (https://pytorch.org/vision/stable/_modules/torchvision/datasets/folder.html#ImageFolder)

In [15]:
from torchvision import datasets, transforms

train_dataset = datasets.ImageFolder(DATA_PATH / 'train')
val_dataset = datasets.ImageFolder(DATA_PATH / 'val')

In [17]:
print(f"Liczba obrazów w zbiorze train: {len(train_dataset)}")
print(f"Liczba obrazów w zbiorze val: {len(val_dataset)}")
for i in range(5):
    image, label = train_dataset[i]
    print(f"Obraz {i} - rozmiar: {image.size}")

Liczba obrazów w zbiorze train: 244
Liczba obrazów w zbiorze val: 153
Obraz 0 - rozmiar: (768, 512)
Obraz 1 - rozmiar: (500, 333)
Obraz 2 - rozmiar: (500, 282)
Obraz 3 - rozmiar: (500, 335)
Obraz 4 - rozmiar: (500, 348)


In [18]:
print(f"Klasy w zbiorze danych: {train_dataset.classes}")
print(f"Mapowanie klas na indeksy: {train_dataset.class_to_idx}")
print(f"Ścieżki do plików obrazów: {train_dataset.imgs[:5]}")

Klasy w zbiorze danych: ['ants', 'bees']
Mapowanie klas na indeksy: {'ants': 0, 'bees': 1}
Ścieżki do plików obrazów: [('common\\data\\hymenoptera_data\\train\\ants\\0013035.jpg', 0), ('common\\data\\hymenoptera_data\\train\\ants\\1030023514_aad5c608f9.jpg', 0), ('common\\data\\hymenoptera_data\\train\\ants\\1095476100_3906d8afde.jpg', 0), ('common\\data\\hymenoptera_data\\train\\ants\\1099452230_d1949d3250.jpg', 0), ('common\\data\\hymenoptera_data\\train\\ants\\116570827_e9c126745d.jpg', 0)]


#### Ćwiczenie

Domyślnie `ImageFolder` dataset przechowuje obrazki jako `PIL.Image`. Należy je przekształcić do tensorów, aby ich użyć w treningu modeli.

Odszukaj odpowiednią funkcję z `torchvision.transforms` (https://pytorch.org/vision/stable/transforms.html) i ponownie wczytaj zbiory danych z jej wykorzystaniem.

In [19]:
data_transforms = transforms.Compose([
    transforms.ToTensor()           
])

train_dataset = datasets.ImageFolder(DATA_PATH / 'train', transform=data_transforms)
val_dataset = datasets.ImageFolder(DATA_PATH / 'val', transform=data_transforms)


#### Ćwiczenie
W kolejnym kroku należy znormalizować wejściowe obrazki. Ponownie odszukaj odpowiednią transformację w `torchvision.transforms` i zbuduj listy transformacji `train_transforms` i `valid_transforms` z użyciem `transforms.Compose`. Wykorzystaj odpowiednie informacje z poniższej komórki.

In [21]:
# średnie i odchylenia standardowe dla kanałów RGB dla zbioru uczącego ImageNet
# ciekawostka: https://github.com/pytorch/vision/issues/1439
IMAGENET_MEANS = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

In [23]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),          
    transforms.Normalize(mean=IMAGENET_MEANS, std=IMAGENET_STD)  
])

train_dataset = datasets.ImageFolder(DATA_PATH / 'train', transform=train_transforms)
val_dataset = datasets.ImageFolder(DATA_PATH / 'val', transform=train_transforms)

#### Ćwiczenie

Należy także doprowadzić oryginalne obrazki do odpowiednich wymiarów (224 na 224 piksele).

W przypadku sieci (pre)trenowanych na danych ImageNet przyjęło się robić to dwukrokowo:
- "resize" obrazka, aby krótszy wymiar miał długość 256
- przycięcie ("crop") obrazka do jego środkowej części 224x224

Rozszerz listę transformacji **podczas walidacji** zgodnie z powyższym opisem, wykorzystując odpowiednie funkcje z https://pytorch.org/vision/stable/transforms.html. Transformacjami treningowymi zajmiemy się w następnym ćwiczeniu.

In [25]:
IMAGENET_IMG_SIZE = 224
IMAGENET_RESIZE = 256

In [26]:
train_transforms = transforms.Compose([
    transforms.Resize(IMAGENET_RESIZE),               
    transforms.CenterCrop(IMAGENET_IMG_SIZE),        
    transforms.ToTensor(),                            
    transforms.Normalize(mean=IMAGENET_MEANS, std=IMAGENET_STD) 
])

train_dataset = datasets.ImageFolder(DATA_PATH / 'train', transform=train_transforms) 
val_dataset = datasets.ImageFolder(DATA_PATH / 'val', transform=train_transforms)


#### Ćwiczenie

Podczas treningu warto - zwłaszcza w przypadku posiadania niewielkiego zbioru danych - zastosować tzw. augmentację danych (więcej na kolejnych zajęciach).

Zamiast "sztywnego" resize'owania obrazka i przycinania go względem środka, w czasie treningu:
- dokonaj "resize" do losowej skali, a następnie przytnij do (losowego) fragmentu 224x224
- dodatkowo losowo (domyślnie: prawdopodobieństwo 50%) przerzuć obrazek względem osi pionowej

Znajdź odpowiednie funkcje w https://pytorch.org/vision/stable/transforms.html.
Stwórz w ten sposób listę transformacji `train_transforms`.

Wczytaj ponownie zbiór uczący i walidacyjny, podając odpowiednie listy transformacji do `ImageFolder`.

In [27]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGENET_IMG_SIZE),  
    transforms.RandomHorizontalFlip(), 
    transforms.CenterCrop(IMAGENET_IMG_SIZE),                 
    transforms.ToTensor(),                             
    transforms.Normalize(mean=IMAGENET_MEANS, std=IMAGENET_STD)  
])

In [38]:
train_dataset = datasets.ImageFolder(DATA_PATH / 'train', transform=train_transforms)
val_dataset = datasets.ImageFolder(DATA_PATH / 'val', transform=train_transforms)


### Transfer learning

Fine-tuningu modeli można dokonać na dwa główne sposoby:
- dotrenować (optymalizować) wszystkie parametry (we wszystkich warstwach) pretrenowanego modelu
- "zamrozić" pretrenowaną część modelu i dotrenować

Na początek zajmiemy się pierwszym z wymienionych sposobów transfer learningu.

#### Ćwiczenie

Załaduj pretrenowaną sieć `resnet18` do zmiennej `model` i dostosuj ją do rozważanego problemu (co musisz zrobić?).
Następnie uruchom trening modelu.

In [36]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    start_time = time.time()
    
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            
        epoch_loss = loss_train / len(train_loader)
        if epoch == 1 or epoch % 5 == 0:
            print(f"Epoch {epoch}, Training loss {epoch_loss}")
            
    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [53]:
import torch.nn as nn
import time
from torch.utils.data import DataLoader

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
model = models.resnet18(pretrained=True)
num_classes = 2
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# training_loop(n_epochs=10, optimizer=optimizer, model=model, loss_fn=loss_fn, train_loader=train_loader)

In [54]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# SGD with momentum
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 25,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 0.696533739566803
Epoch 5, Training loss 0.11297923792153597
Epoch 10, Training loss 0.06457358552142978
Epoch 15, Training loss 0.05301452940329909
Epoch 20, Training loss 0.04594590375199914
Epoch 25, Training loss 0.08516848087310791
Training complete in 1m 13s


#### Ćwiczenie

Sprawdź jakość wytrenowanego modelu uruchamiając poniższe komórki.

In [55]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

In [56]:
def validate(model, train_loader, val_loader):
    model.eval()
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device)
                labels = labels.to(device)
                outputs = model(imgs)
                preds = torch.argmax(outputs, dim=1)
                total += labels.shape[0]
                correct += int((preds == labels).sum())
                
        print(f"{name} accuracy: {correct/total}")

In [57]:
validate(model, train_loader, val_loader)

train accuracy: 0.9549180327868853
val accuracy: 0.9150326797385621


#### Ćwiczenie

Jeszcze raz załaduj i przygotuj model `resnet18` (np. do zmiennej `model_frozen`, tym razem "zamrażając" wszystkie pretrenowane warstwy modelu.
Wytrenuj model i sprawdź jego dokładność.

In [46]:
model_frozen = models.resnet18(pretrained=True)

for param in model_frozen.parameters():
    param.requires_grad = False

num_classes = 2
num_ftrs = model_frozen.fc.in_features
model_frozen.fc = nn.Linear(num_ftrs, num_classes)

model_frozen = model_frozen.to(device)

In [47]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# SGD with momentum
optimizer = torch.optim.SGD(model_frozen.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 25,
    optimizer = optimizer,
    model = model_frozen,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 0.6365495771169662
Epoch 5, Training loss 0.16748459078371525
Epoch 10, Training loss 0.12793205678462982
Epoch 15, Training loss 0.09340222552418709
Epoch 20, Training loss 0.07861949596554041
Epoch 25, Training loss 0.13101957738399506
Training complete in 0m 48s


In [48]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

In [49]:
validate(model_frozen, train_loader, val_loader)

train accuracy: 0.9713114754098361
val accuracy: 0.9215686274509803


#### Ćwiczenie

Porównaj powyższe wyniki z uzyskanymi dla modelu `Net` stworzonego na wcześniejszych zajęciach (lekko zmodyfikowane wymiary dla warstw gęstych - inny rozmiar obrazków wejściowych).

In [50]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 56 * 56, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 56 * 56)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [51]:
net_model = Net()
net_model = net_model.to(device)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# SGD with momentum
optimizer = torch.optim.SGD(net_model.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 25,
    optimizer = optimizer,
    model = net_model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 0.6748300045728683
Epoch 5, Training loss 0.6375532001256943
Epoch 10, Training loss 0.6169845461845398
Epoch 15, Training loss 0.6241620928049088
Epoch 20, Training loss 0.6095932871103287
Epoch 25, Training loss 0.5838529914617538
Training complete in 0m 35s


In [52]:
validate(net_model, train_loader, val_loader)

train accuracy: 0.6721311475409836
val accuracy: 0.6339869281045751
